## Walmart sales - data preprocessing

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

from IPython.display import display

### Inputs

#### data/raw/train.csv

This is the historical training data, which covers from 2010-02-05 to 2012-11-01, with the following fields:

* Store - the store number
* Dept - the department number (should be discarded, and sales should be averaged by store)
* Date - the week
* Weekly_Sales -  sales for the given department in the given store
* IsHoliday - whether the week is a special holiday week

#### data/raw/test.csv

This file is identical to train.csv, except it doesn't contain weekly sales. Predictions should be made for each triplet of store and date in this file.

For convenience, the four holidays fall within the following weeks in the dataset (not all holidays are in the data):

* Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
* Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
* Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
* Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

### Outputs

#### data/processed/all.csv

This file will hold all the training data from 2010-02-05 to 2012-11-01 aggregated by store, and the prediction placeholders from 2012-11-02 and 2013-07-26, with the following fields:

* Store - the store number
* Dept - the department number
* Date - the week (YYYY-MM-DD format, datetime)
* DateString - the week (YYYY-MM-DD format, string)
* Year - the year
* Week - week within the year
* YearWeek - year-week composition
* WeeklySales -  sales for the given department in the given store
* WeeklySalesPreviousYear - previous year sales for the same week
* WeeklySalesIncreaseRatio - ratio between sales from this year and previous year for the same week
* IsHoliday - whether the week is a special holiday week (boolean)
* IsHolidayInt - whether the week is a special holiday week (integer)
* Store_* - dummy variables for store


Let's first build a single dataframe with train and prediction dates so that any feature processing can be performed consistently

In [2]:
df_train = pd.read_csv('data/raw/train.csv')

# aggregate data per store and flag rows to differentiate from prediction placeholders
df_train = df_train.rename(columns={'Weekly_Sales': 'WeeklySales'})
df_train = df_train.groupby(['Store','Date','IsHoliday'])['WeeklySales'].agg(np.mean).to_frame().reset_index()
df_train.WeeklySales = df_train.WeeklySales.map(int)
df_train['Train'] = True

# predicition placeholders
df_pred = pd.read_csv('data/raw/test.csv').drop(['Dept'], axis=1).drop_duplicates()
df_pred['Train'] = False

df = pd.concat([df_train, df_pred])
df.sort_values(by=['Date','Store'], inplace=True)
df.reset_index(drop=True, inplace=True)

print("train: %s %s" % (df_train.Date.min(), df_train.Date.max()))
print("pred: %s %s" % (df_pred.Date.min(), df_pred.Date.max()))

del(df_train)
del(df_pred)

display(df.head())
display(df.tail())

train: 2010-02-05 2012-10-26
pred: 2012-11-02 2013-07-26


,Date,IsHoliday,Store,Train,WeeklySales
0,2010-02-05,False,1,True,22516.0
1,2010-02-05,False,2,True,29273.0
2,2010-02-05,False,3,True,7101.0
3,2010-02-05,False,4,True,29248.0
4,2010-02-05,False,5,True,5115.0


,Date,IsHoliday,Store,Train,WeeklySales
8185,2013-07-26,False,41,False,NaN
8186,2013-07-26,False,42,False,NaN
8187,2013-07-26,False,43,False,NaN
8188,2013-07-26,False,44,False,NaN
8189,2013-07-26,False,45,False,NaN


Break dates into year, month, day and week

In [3]:
df['DateString'] = df['Date']
df['Date'] = pd.to_datetime(df['Date'])
df['Week'] = df['Date'].dt.week
df['Year'] = df['Date'].dt.year
df['YearWeek'] = df.Year.map(str) + "-" + df.Week.map(lambda v: "%02d"%v)
df.head(5)

,Date,IsHoliday,Store,Train,WeeklySales,DateString,Week,Year,YearWeek
0,2010-02-05,False,1,True,22516.0,2010-02-05,5,2010,2010-05
1,2010-02-05,False,2,True,29273.0,2010-02-05,5,2010,2010-05
2,2010-02-05,False,3,True,7101.0,2010-02-05,5,2010,2010-05
3,2010-02-05,False,4,True,29248.0,2010-02-05,5,2010,2010-05
4,2010-02-05,False,5,True,5115.0,2010-02-05,5,2010,2010-05


Creating a new column with the sales increase from previous year

In [4]:
df['WeeklySalesPreviousYear'] = np.nan

for store in df.Store.unique():
    idx = df.Store==store
    a = df['WeeklySales'][idx].values
    a = np.roll(a,52) # shift by one year
    a[0:52] = np.nan
    df.loc[idx, 'WeeklySalesPreviousYear'] = a
    
df['WeeklySalesIncreaseRatio'] = df['WeeklySales'] / df['WeeklySalesPreviousYear']

Creating dummy variables for Store, and mapping IsHoliday to integers

In [5]:
stores = df.Store
df = pd.get_dummies(df, columns=['Store'])
df['Store'] = stores
df['IsHolidayInt'] = df.IsHoliday.map(int)

df.sample(5)

,Date,IsHoliday,Train,WeeklySales,DateString,Week,Year,YearWeek,WeeklySalesPreviousYear,WeeklySalesIncreaseRatio,...,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45,Store,IsHolidayInt
6840,2013-01-04,False,False,NaN,2013-01-04,1,2013,2013-01,21532.0,NaN,...,0,0,0,0,0,0,0,0,1,0
1564,2010-10-01,False,True,11508.0,2010-10-01,39,2010,2010-39,NaN,NaN,...,0,0,0,0,0,0,0,0,35,0
7799,2013-05-31,False,False,NaN,2013-05-31,22,2013,2013-22,9622.0,NaN,...,0,0,0,0,0,0,0,0,15,0
32,2010-02-05,False,True,6864.0,2010-02-05,5,2010,2010-05,NaN,NaN,...,0,0,0,0,0,0,0,0,33,0
2061,2010-12-17,False,True,10903.0,2010-12-17,50,2010,2010-50,NaN,NaN,...,0,0,0,0,0,0,0,0,37,0


### Persisting dataframes

In [6]:
df.to_csv('data/processed/all.csv')